In [20]:
import os
import joblib
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import numpy as np

In [2]:
def load_feats(path, name):
    with open(os.path.join(path, str.join('', (name, '_feats.sav'))), 'rb') as fr:
        data = joblib.load(fr)
    return [i for i in data]


def load_clusters(path, name):
    with open(os.path.join(path, str.join('', (name, '_clusters.sav'))), 'rb') as fr:
        data = joblib.load(fr)
    return [i for i in data]

In [3]:
path = 'D:/Jack/Project/output/'
name = 'Sep-07-2022'
features, scaled_features = load_feats(path, name)

In [5]:
[_, assignments, _, soft_assignments] = load_clusters(path, name)

In [25]:
features_df = pd.DataFrame(features.T[assignments>=0, :])
features_df

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,28.251890,37.353091,74.132391,97.581141,119.250010,28.810101,46.219590,78.472473,93.500601,53.565737,...,-0.370150,-0.980581,-0.835089,-1.472400,4.836097,2.811972,1.994454,0.777261,0.859837,1.432244
1,17.660453,40.345334,55.111148,92.610339,104.876809,24.998461,43.785940,75.737268,90.486857,51.170218,...,7.305572,0.703636,4.959827,2.077367,16.221747,2.360729,8.419852,1.661657,2.494970,5.345820
2,41.260998,33.104164,67.603430,94.890587,113.251746,23.769351,32.349420,54.682663,72.875555,55.824477,...,5.202205,-21.012716,-5.874874,-0.072087,13.910119,7.588711,10.402583,21.608048,4.369871,11.745376
3,38.410512,39.488726,67.150482,86.467953,114.531206,25.420594,30.183348,54.750079,76.802259,44.322776,...,5.199740,36.743331,8.572212,20.096432,7.989317,29.319428,3.405875,7.109486,34.457089,13.923869
4,25.191220,32.773055,86.794620,73.072302,116.195550,12.820155,62.239943,52.313367,91.875073,58.980104,...,-9.742632,0.885024,2.956705,-9.581491,26.577718,34.821585,20.108117,3.965679,4.547231,12.715968
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33830,60.232202,25.611382,69.665247,25.015322,77.917309,35.860028,46.159767,50.366972,87.035698,48.393428,...,-1.502727,3.789255,-3.160873,0.049830,12.987166,3.011070,3.598596,1.696568,8.391045,3.825294
33831,63.167394,36.675551,81.643533,40.335489,84.064737,27.178552,33.943540,42.811808,69.724225,47.519219,...,-13.077600,-5.265384,-5.920696,-14.812852,19.990315,28.336750,3.495055,7.638469,2.458407,12.856823
33832,64.018438,43.772505,93.217416,80.174845,106.017654,32.674521,38.896723,36.583068,63.123134,51.396357,...,-24.714242,-82.282710,-78.939645,-32.088993,22.733418,28.508055,10.364843,65.665006,3.023926,21.981141
33833,58.902005,59.474777,94.136296,74.244998,107.256925,19.851812,36.582906,36.343877,59.456353,38.882895,...,-2.200982,-6.391898,-7.120537,0.486852,11.306153,13.501843,8.590168,13.903899,4.480267,6.603114


In [24]:
assign_df = pd.DataFrame(assignments[assignments>=0])
assign_df

,0
0,10
1,10
2,17
3,12
4,18
...,...
33830,12
33831,8
33832,12
33833,10


In [40]:
assign_df.iloc[np.where(assign_df != 17)[0]] = 0

In [43]:
np.unique(assign_df)

array([ 0, 17], dtype=int64)

In [59]:
sampled_idx_of_interest = np.where(assign_df>0)[0]
size_of_target =len(sampled_idx_of_interest)
size_of_target

2104

In [60]:
sampled_idx_of_noninterest = np.random.choice(np.where(assign_df == 0)[0], size_of_target, replace=False)
sampled_idx_of_noninterest.shape

(2104,)

In [71]:
X, y = features_df, assign_df
X_sub, y_sub = np.vstack([X.iloc[sampled_idx_of_interest, :], X.iloc[sampled_idx_of_noninterest, :]]), np.vstack([y.iloc[sampled_idx_of_interest], y.iloc[sampled_idx_of_noninterest]])
random_forest_clf = RandomForestClassifier(random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_sub, y_sub, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape)

(3366, 36) (842, 36)


In [72]:
np.unique(y_train), np.unique(y_test)

(array([ 0, 17], dtype=int64), array([ 0, 17], dtype=int64))

In [73]:
random_forest_clf.fit(X_train, y_train)

C:\Users\jimi\AppData\Local\Temp\ipykernel_14540\2185478983.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  random_forest_clf.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [74]:
y_predict = random_forest_clf.predict(X_test)
print(y_predict.shape)

(842,)


In [75]:
np.unique(y_predict)

array([ 0, 17], dtype=int64)

In [76]:
f1_score(y_test, y_predict, average=None)

array([0.53095238, 0.53317536])